In [1]:
!pip install tensorflow pandas scikit-learn



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.optimizers import Adam
from keras.losses import Huber
from tensorflow.keras.callbacks import EarlyStopping

pd.set_option('display.max_columns',None)

In [3]:
def call_rmsle(y_true, y_pred):
    # Ensure predictions and true values are positive by adding 1
    y_true = tf.math.maximum(y_true, 0)
    y_pred = tf.math.maximum(y_pred, 0)

    # Compute the logarithm of the true and predicted values
    log_true = tf.math.log1p(y_true)
    log_pred = tf.math.log1p(y_pred)

    # Compute the squared difference
    squared_diff = tf.math.square(log_true - log_pred)

    # Compute the mean of the squared differences
    mean_squared_diff = tf.math.reduce_mean(squared_diff)

    # Return the square root of the mean squared differences
    return tf.math.sqrt(mean_squared_diff)

def remove_leading_zeros(df):
    # Find the index of the first non-zero sales value
    first_non_zero_index = df['sales'].ne(0).idxmax()

    # Filter the DataFrame to remove leading zeros
    return df.loc[first_non_zero_index:]

In [4]:
df_train_all    = pd.read_csv('data_dir/wip_data/preprocessed_train_data_v1.csv',index_col=0)


In [5]:
df_test_all    = pd.read_csv('data_dir/wip_data/preprocessed_test_data_v1.csv',index_col=0)

In [6]:
df_lstm_results  = pd.DataFrame()

In [7]:
all_store_nr_all   = df_train_all['store_nbr'].unique()
all_store_nr_all   = np.sort(all_store_nr_all)

In [8]:
all_store_nr       = all_store_nr_all[47:54]

In [9]:
for store_nr in (all_store_nr):
    all_family =   df_train_all[df_train_all['store_nbr']==store_nr]['family'].unique()  
    for fam in all_family:
        if df_train_all[(df_train_all['store_nbr']==store_nr) & (df_train_all['family']==fam)]['sales'].sum() > 0 :
            print("Running for store ", store_nr," and family ",fam)
            
            df_train    = df_train_all[(df_train_all['store_nbr']==store_nr) & (df_train_all['family']==fam)]
            df_test     = df_test_all[(df_test_all['store_nbr']==store_nr) & (df_test_all['family']==fam)]
            ###############################################################################
            ##########       Some Preprocessing is required     ###########################
            ###############################################################################

            df  = df_train.copy()

            #removing starting sales = 0 rows
            df = remove_leading_zeros(df)

            # Convert month and day to cyclical features
            df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
            df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
            df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
            df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)

            df['wom_sin'] = np.sin(2 * np.pi * df['week_of_month'] / 5)
            df['wom_cos'] = np.cos(2 * np.pi * df['week_of_month'] / 5)

            df['week_sin'] = np.sin(2 * np.pi * df['week_number'] / 52)
            df['week_cos'] = np.cos(2 * np.pi * df['week_number'] / 52)


            df['is_weekend'] = df['day_of_week'].isin([0, 6]).astype('int')
            df['is_midweek'] = df['day_of_week'].isin([3, 4]).astype('int')


            df['year'] = df['year']-df['year'].min()

            ###########################################################################################
            ###############     Preparing for LSTM              ######################################
            #########################################################################################

            # Features to be scaled
            features_to_scale = ['holiday_type', 'oil_price', 'year']
            # Features to be left unscaled (those ending with 'sin' or 'cos')
            features_to_leave = ['onpromotion', 'IS_HOLIDAY',  'month_sin','month_cos',
                     'day_sin','day_cos',  'wom_sin','wom_cos', 'week_sin','week_cos','is_weekend','is_midweek' ]

            features = features_to_scale + features_to_leave
            target = 'sales'

            # Splitting data into training and testing sets
            X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, shuffle=False)

            # Initialize scalers
            feature_scaler = MinMaxScaler(feature_range=(0, 1))
            target_scaler = MinMaxScaler(feature_range=(0, 1))

            # Scale only the selected features
            X_train_scaled_part = feature_scaler.fit_transform(X_train[features_to_scale])
            X_test_scaled_part = feature_scaler.transform(X_test[features_to_scale])

            # Combine scaled features with unscaled features (still as NumPy arrays)
            X_train_scaled = np.hstack((X_train_scaled_part, X_train[features_to_leave].values))
            X_test_scaled = np.hstack((X_test_scaled_part, X_test[features_to_leave].values))

            # Scale target variable
            y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1, 1))
            y_test_scaled = target_scaler.transform(y_test.values.reshape(-1, 1))

            #############################################################################################
            #################                Train the model                         ####################
            #############################################################################################

            # Reshape data to fit LSTM input shape (num_samples, timesteps, num_features)
            timesteps = 1  # You need to define this based on your data and model requirements
            X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
            X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))


            # Build your LSTM model
            model = Sequential()
            model.add(LSTM(256, return_sequences=True, input_shape=(timesteps, X_train_reshaped.shape[2])))
            model.add(Dropout(0.2))
            model.add(LSTM(100))
            model.add(Dropout(0.2))

            model.add(Dense(1))

            # Define early stopping
            early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

            # Compile the model with RMSLE as the loss function
            optimizer = Adam(learning_rate=0.001)
            model.compile(optimizer=optimizer, loss=call_rmsle,metrics = ['accuracy'] )

            # Train the model
            history = model.fit(X_train_reshaped, y_train_scaled, epochs=30, 
                                validation_data=(X_test_reshaped, y_test_scaled), batch_size=32,callbacks=[early_stopping])

            ##############################################################################################
            #########################          Make Predictions              #############################
            ##############################################################################################


            # Predictions
            lstm_predictions = model.predict(X_test_reshaped)

            # Ensure both predictions and actual values are of type float64
            y_test_scaled = y_test_scaled.reshape(-1, 1).astype('float64')
            lstm_predictions_scaled = lstm_predictions.astype('float64')

            # Calculate RMSLE
            lstm_rmsle_val = call_rmsle(y_test_scaled, lstm_predictions_scaled)

            y_test_actual = target_scaler.inverse_transform(y_test_scaled).astype('float64')
            lstm_predictions_actual = target_scaler.inverse_transform(lstm_predictions_scaled).astype('float64')
            lstm_rmsle_val_actual = call_rmsle(y_test_actual, lstm_predictions_actual)
            print("RSMLE we got for store number ",store_nr," and family ",fam," is ",lstm_rmsle_val_actual)

            ##############################################################################################
            #########################             Applying model on test data       ######################
            ##############################################################################################

            del df
            df   = df_test.copy()

            ###############################################################################
            ##########       test_data_preprocessing            ###########################
            ###############################################################################



            # Convert month and day to cyclical features
            df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
            df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
            df['day_sin'] = np.sin(2 * np.pi * df['day'] / 31)
            df['day_cos'] = np.cos(2 * np.pi * df['day'] / 31)

            df['wom_sin'] = np.sin(2 * np.pi * df['week_of_month'] / 5)
            df['wom_cos'] = np.cos(2 * np.pi * df['week_of_month'] / 5)

            df['week_sin'] = np.sin(2 * np.pi * df['week_number'] / 52)
            df['week_cos'] = np.cos(2 * np.pi * df['week_number'] / 52)

            df['is_weekend'] = df['day_of_week'].isin([0, 6]).astype('int')
            df['is_midweek'] = df['day_of_week'].isin([3, 4]).astype('int')

            df['year'] = df['year']-df_train['year'].min()

            ###########################################################################################
            ###############     Preparing for LSTM              ######################################
            #########################################################################################

            # Features to be scaled
            features_to_scale = ['holiday_type', 'oil_price', 'year']
            # Features to be left unscaled (those ending with 'sin' or 'cos')
            features_to_leave = ['onpromotion', 'IS_HOLIDAY',  'month_sin','month_cos',
                     'day_sin','day_cos',  'wom_sin','wom_cos', 'week_sin','week_cos','is_weekend','is_midweek' ]

            features = features_to_scale + features_to_leave

            # Scale only the selected features
            X_test_scaled_part = feature_scaler.transform(df[features_to_scale])

            # Combine scaled features with unscaled features (still as NumPy arrays)
            X_test_scaled = np.hstack((X_test_scaled_part, df[features_to_leave].values))

            X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

            # Predictions
            lstm_predictions = model.predict(X_test_reshaped)
            lstm_predictions_scaled = lstm_predictions.astype('float64')

            lstm_predictions_actual = target_scaler.inverse_transform(lstm_predictions_scaled).astype('float64')

            # Add predictions to the dataframe
            df['predicted_sales'] = lstm_predictions_actual

            this_df   = df[['id','predicted_sales']]

            df_lstm_results = pd.concat([df_lstm_results,this_df])

            del this_df
            del df

Running for store  48  and family  0


C:\Users\Monil\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.0032 - loss: 0.1639 - val_accuracy: 0.0089 - val_loss: 0.0977
Epoch 2/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0019 - loss: 0.0954 - val_accuracy: 0.0089 - val_loss: 0.0887
Epoch 3/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.0013 - loss: 0.0956 - val_accuracy: 0.0089 - val_loss: 0.0976
Epoch 4/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.0037 - loss: 0.0917 - val_accuracy: 0.0089 - val_loss: 0.0942
Epoch 5/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0024 - loss: 0.0883 - val_accuracy: 0.0089 - val_loss: 0.0931
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
RSMLE we got for store number  48  and family  0  is  tf.Tensor(0.5726870739387513, shape=(), dtype=float64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Running for store  48  and family  2
Epoch 1/30


C:\Users\Monil\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.0078 - loss: 0.1484 - val_accuracy: 0.0030 - val_loss: 0.1241
Epoch 2/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0068 - loss: 0.0846 - val_accuracy: 0.0000e+00 - val_loss: 0.1243
Epoch 3/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0133 - loss: 0.0814 - val_accuracy: 0.0030 - val_loss: 0.1271
Epoch 4/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0093 - loss: 0.0779 - val_accuracy: 0.0030 - val_loss: 0.1250
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 35ms/step
RSMLE we got for store number  48  and family  2  is  tf.Tensor(0.4803695042240014, shape=(), dtype=float64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Running for store  48  and family  3
Epoch 1/30


...

Epoch 1/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 25s 62ms/step - accuracy: 0.2685 - loss: 0.2196 - val_accuracy: 0.0038 - val_loss: 0.1048
Epoch 2/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.2784 - loss: 0.1221 - val_accuracy: 0.0038 - val_loss: 0.0815
Epoch 3/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.3090 - loss: 0.1268 - val_accuracy: 0.0000e+00 - val_loss: 0.0792
Epoch 4/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2820 - loss: 0.1221 - val_accuracy: 0.0000e+00 - val_loss: 0.0777
Epoch 5/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.2866 - loss: 0.1187 - val_accuracy: 0.0000e+00 - val_loss: 0.0798
Epoch 6/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.2798 - loss: 0.1166 - val_accuracy: 0.0000e+00 - val_loss: 0.0834
Epoch 7/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.2692 - loss: 0.1118 - val_accuracy: 0.0038 - val_loss: 0.0770
Epoch 8/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.2810 - loss: 0.1182 - val_acc

C:\Users\Monil\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 13s 87ms/step - accuracy: 0.8739 - loss: 0.0405 - val_accuracy: 0.7544 - val_loss: 0.0870
Epoch 2/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.8768 - loss: 0.0367 - val_accuracy: 0.7544 - val_loss: 0.0827
Epoch 3/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.8896 - loss: 0.0252 - val_accuracy: 0.7544 - val_loss: 0.0850
Epoch 4/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8781 - loss: 0.0374 - val_accuracy: 0.7544 - val_loss: 0.0815
Epoch 5/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8744 - loss: 0.0315 - val_accuracy: 0.7544 - val_loss: 0.0798
Epoch 6/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.8977 - loss: 0.0280 - val_accuracy: 0.7544 - val_loss: 0.0849
Epoch 7/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8834 - loss: 0.0254 - val_accuracy: 0.7544 - val_loss: 0.0895
Epoch 8/30
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8828 - loss: 0.0315 - val_accuracy: 0.7544 - 

C:\Users\Monil\anaconda3\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.4179 - loss: 0.0844 - val_accuracy: 0.3145 - val_loss: 0.0775
Epoch 2/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4057 - loss: 0.0763 - val_accuracy: 0.3145 - val_loss: 0.0832
Epoch 3/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4082 - loss: 0.0775 - val_accuracy: 0.3145 - val_loss: 0.0802
Epoch 4/30
43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4129 - loss: 0.0811 - val_accuracy: 0.3145 - val_loss: 0.0793
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step
RSMLE we got for store number  54  and family  32  is  tf.Tensor(0.6861074533301718, shape=(), dtype=float64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


In [10]:
df_lstm_results.to_csv('data_dir/output_data/lstm_prediction_submission_47_54.csv',index=False)